# Open Library - File ingestion and schema creation in Databricks

In [0]:
# aws_access_key_id= dbutils.widgets.get("access_key")
# aws_secret_access_key= dbutils.widgets.get("secret_access_key")
# encoded_secret_key = aws_secret_access_key.replace("/", "%2F")
# bucket_name = dbutils.widgets.get("s3_bucket")
# mount_name = "cidaen-open-library"

# dbutils.fs.mount("s3a://%s:%s@%s" % (aws_access_key_id, encoded_secret_key, bucket_name), "/mnt/%s" % mount_name)
# display(dbutils.fs.ls("/mnt/%s" % mount_name))

### Connection setup with Data Lake

__Steps:__
Create App registration: Go to Azure AD >> App Registration and register an app. We need the client ID and the tenant ID
Create Secret: In the Registered App, create a Secret and store it.
Role assignment: In the Storage account that we want to connect to, create a role assignment of Contributor or similar with read/write access for Blob storage.

## Bronze

#### Imports

In [0]:
from pyspark.sql.functions import current_date, current_timestamp, lit

### Create tables from files available in ADLS2

The folder contains compressed files.

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze_openlibrary

In [0]:
def create_df_ol_table(source_file):
    # This function generates a df from the file (ADL2) passed to it as argument:
    df = spark.read.csv(f"dbfs:/mnt/cidaen-open-library/{source_file}",sep=r'\t', header=False, inferSchema=True)
    return df

#### Create table bronze_openlibrary.ol_ratings

In [0]:
df = create_df_ol_table("ol_dump_ratings_2023-01-10.txt.gz")

In [0]:
df.display()

In [0]:
df = (df.withColumnRenamed("_c0","reference")
  .withColumnRenamed("_c1","reference_b")
  .withColumnRenamed("_c2","rating")
  .withColumnRenamed("_c3","review_date")
  .withColumn("ctl_created_timestamp", current_timestamp().cast("string"))
  .withColumn("ctl_source", lit("historical_load"))
)

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView("tempvw_ratings")

In [0]:
%sql
CREATE OR REPLACE TABLE bronze_openlibrary.ol_ratings AS
SELECT DISTINCT *, input_file_name() AS ctl_file_name  
FROM tempvw_ratings;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM bronze_openlibrary.ol_ratings;

count(1)
321435


In [0]:
%sql
SELECT * FROM bronze_openlibrary.ol_ratings LIMIT 5;

reference,reference_b,rating,review_date,ctl_created_timestamp,ctl_source,ctl_file_name
/works/OL8032382W,/books/OL10691390M,2,2018-06-21,2023-06-16 18:16:26.107,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_ratings_2023-01-10.txt.gz
/works/OL16716976W,/books/OL25386811M,5,2018-06-25,2023-06-16 18:16:26.107,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_ratings_2023-01-10.txt.gz
/works/OL10530589W,null,2,2018-06-26,2023-06-16 18:16:26.107,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_ratings_2023-01-10.txt.gz
/works/OL3519317W,/books/OL10738407M,4,2018-07-02,2023-06-16 18:16:26.107,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_ratings_2023-01-10.txt.gz
/works/OL3657595W,null,4,2018-07-15,2023-06-16 18:16:26.107,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_ratings_2023-01-10.txt.gz


In [0]:
df.printSchema()

root
 |-- reference: string (nullable = true)
 |-- reference_b: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_date: date (nullable = true)
 |-- ctl_created_timestamp: string (nullable = false)
 |-- ctl_source: string (nullable = false)



#### Create table bronze_openlibrary.ol_readings

In [0]:
df = create_df_ol_table("ol_dump_reading-log_2023-01-10.txt.gz")

In [0]:
df.display()

In [0]:
df = (df.withColumnRenamed("_c0","reference")
  .withColumnRenamed("_c1","reference_b")
  .withColumnRenamed("_c2","read_status")
  .withColumnRenamed("_c3","last_updated")
  .withColumn("ctl_created_timestamp", current_timestamp().cast("string"))
  .withColumn("ctl_source", lit("historical_load"))
)

In [0]:
df.createOrReplaceTempView("tempvw_readings")

In [0]:
%sql
CREATE OR REPLACE TABLE bronze_openlibrary.ol_readings AS
SELECT DISTINCT *, input_file_name() AS ctl_file_name FROM tempvw_readings;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM bronze_openlibrary.ol_readings;

count(1)
4882308


In [0]:
%sql
SELECT * FROM bronze_openlibrary.ol_readings LIMIT 5;

reference,reference_b,read_status,last_updated,ctl_created_timestamp,ctl_source,ctl_file_name
/works/OL9693711W,/books/OL19016795M,Want to Read,2017-11-15,2023-06-16 18:17:32.728,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_reading-log_2023-01-10.txt.gz
/works/OL7322980W,null,Want to Read,2017-12-27,2023-06-16 18:17:32.728,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_reading-log_2023-01-10.txt.gz
/works/OL8366901W,/books/OL8748118M,Want to Read,2018-01-22,2023-06-16 18:17:32.728,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_reading-log_2023-01-10.txt.gz
/works/OL8004906W,null,Want to Read,2018-03-15,2023-06-16 18:17:32.728,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_reading-log_2023-01-10.txt.gz
/works/OL3498685W,null,Want to Read,2017-12-27,2023-06-16 18:17:32.728,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_reading-log_2023-01-10.txt.gz


type - type of record (/type/edition, /type/work etc.) </br>
key - unique key of the record. (/books/OL1M etc.) </br>
revision - revision number of the record </br>
last_modified - last modified timestamp </br>
JSON - the complete record in JSON format </br>

#### Create table bronze_openlibrary.ol_works

In [0]:
df = create_df_ol_table("ol_dump_works_2023-01-10.txt.gz")

In [0]:
df.display()

In [0]:
df = (df.withColumnRenamed("_c0","type")
  .withColumnRenamed("_c1","reference")
  .withColumnRenamed("_c2","revision_number")
  .withColumnRenamed("_c3","last_modified")
  .withColumnRenamed("_c4","json_col")
  .withColumn("ctl_created_timestamp", current_timestamp().cast("string"))
  .withColumn("ctl_source", lit("historical_load"))
)

In [0]:
df.createOrReplaceTempView("tempvw_works")

In [0]:
%sql
CREATE OR REPLACE TABLE bronze_openlibrary.ol_works AS
SELECT *, input_file_name() AS ctl_file_name FROM tempvw_works;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * 
FROM bronze_openlibrary.ol_works
LIMIT 5

type,reference,revision_number,last_modified,json_col,ctl_created_timestamp,ctl_source,ctl_file_name
/type/work,/works/OL10000077W,7,2021-08-13T09:02:17.282+0000,"{""title"": ""Classic garden style"", ""subjects"": [""Floral decorations"", ""Garden ornaments and furniture"", ""Decoration and ornament"", ""Garden structures"", ""Jardins"", ""Meubles et ornements"", ""Antiquit\u00e9s (Objets anciens)""], ""key"": ""/works/OL10000077W"", ""authors"": [{""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL3965042A""}}], ""type"": {""key"": ""/type/work""}, ""covers"": [11604950], ""latest_revision"": 7, ""revision"": 7, ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-08-13T09:02:17.282451""}}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000238W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""H\u00e9rault de S\u00e9chelles, ou les infortunes de la beaut\u00e9"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3140482], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000238W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965269A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000261W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""Envoy\u00e9s sp\u00e9cieux"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3142759], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000261W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965289A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000387W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""JC de Castelbajac"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3140687], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000387W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965419A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000507W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""L'Alchimie de la gu\u00e9rison, tome 2"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3140956], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000507W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965546A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz


In [0]:
%sql
SELECT COUNT(*) FROM bronze_openlibrary.ol_works

count(1)
30852119


#### Create table bronze_openlibrary.ol_authors

In [0]:
df = create_df_ol_table("ol_dump_authors_2023-01-10.txt.gz")

In [0]:
df.display()

In [0]:
df = (df.withColumnRenamed("_c0","type")
  .withColumnRenamed("_c1","reference")
  .withColumnRenamed("_c2","revision_number")
  .withColumnRenamed("_c3","last_modified")
  .withColumnRenamed("_c4","json_col")
  .withColumn("ctl_created_timestamp", current_timestamp().cast("string"))
  .withColumn("ctl_source", lit("historical_load"))
)

In [0]:
df.createOrReplaceTempView("tempvw_authors")

In [0]:
%sql
CREATE OR REPLACE TABLE bronze_openlibrary.ol_authors AS
SELECT *, input_file_name() AS ctl_file_name FROM tempvw_authors;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM bronze_openlibrary.ol_authors;

count(1)
11316199


In [0]:
%sql
SELECT * FROM bronze_openlibrary.ol_authors LIMIT 5;

type,reference,revision_number,last_modified,json_col,ctl_created_timestamp,ctl_source,ctl_file_name
/type/author,/authors/OL10000666A,1,2021-12-26T23:05:27.715+0000,"{""type"": {""key"": ""/type/author""}, ""name"": ""Alexander A. Bove"", ""key"": ""/authors/OL10000666A"", ""source_records"": [""bwb:9781466829008""], ""latest_revision"": 1, ""revision"": 1, ""created"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:05:27.715681""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:05:27.715681""}}",2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/type/author,/authors/OL10000704A,1,2021-12-26T23:07:40.545+0000,"{""type"": {""key"": ""/type/author""}, ""name"": ""Angelika Juritsch"", ""key"": ""/authors/OL10000704A"", ""source_records"": [""bwb:9788490154434""], ""latest_revision"": 1, ""revision"": 1, ""created"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:07:40.545377""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:07:40.545377""}}",2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/type/author,/authors/OL10001011A,1,2021-12-26T23:58:07.734+0000,"{""type"": {""key"": ""/type/author""}, ""name"": ""Magnus Molasky"", ""key"": ""/authors/OL10001011A"", ""source_records"": [""bwb:9783850403801""], ""latest_revision"": 1, ""revision"": 1, ""created"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:58:07.734703""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:58:07.734703""}}",2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/type/author,/authors/OL10001044A,1,2021-12-26T23:58:45.859+0000,"{""type"": {""key"": ""/type/author""}, ""name"": ""Angelika Nickel"", ""key"": ""/authors/OL10001044A"", ""source_records"": [""bwb:9788490156315""], ""latest_revision"": 1, ""revision"": 1, ""created"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:58:45.859331""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-12-26T23:58:45.859331""}}",2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/type/author,/authors/OL10001379A,1,2021-12-27T00:53:18.206+0000,"{""type"": {""key"": ""/type/author""}, ""name"": ""Verena ller"", ""key"": ""/authors/OL10001379A"", ""source_records"": [""bwb:9783850405348""], ""latest_revision"": 1, ""revision"": 1, ""created"": {""type"": ""/type/datetime"", ""value"": ""2021-12-27T00:53:18.206332""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-12-27T00:53:18.206332""}}",2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz


## Silver

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver_openlibrary

### Create tables in Silver - First step transformations

#### Imports

In [0]:
from pyspark.sql.functions import from_json, col, regexp_extract, explode, split, collect_list
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, TimestampType, MapType

#### Create table silver_openlibrary.ol_authors

In [0]:
df_authors = spark.sql("SELECT * FROM bronze_openlibrary.ol_authors")
schema = StructType(
    [
        StructField('name', StringType(), True),
        StructField('personal_name', StringType(), True)
    ]
)

df_authors = df_authors.withColumn("json_col", from_json("json_col", schema))\
    .select(col('reference').alias('author_ref'), \
            col('json_col.name').alias('author_name'), \
            col('json_col.personal_name').alias('author_personal_name'), \
            col('revision_number'), \
            col('last_modified'), \
            col('ctl_created_timestamp'), \
            col('ctl_source'), \
            col('ctl_file_name'))
df_authors.createOrReplaceTempView("vw_silver_authors")

In [0]:
%sql
CREATE OR REPLACE TABLE silver_openlibrary.ol_authors AS
SELECT DISTINCT * FROM vw_silver_authors

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM silver_openlibrary.ol_authors LIMIT 5;

author_ref,author_name,author_personal_name,revision_number,last_modified,ctl_created_timestamp,ctl_source,ctl_file_name
/authors/OL10000704A,Angelika Juritsch,null,1,2021-12-26T23:07:40.545+0000,2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/authors/OL10028475A,Néstor A. Corona,null,1,2021-12-28T05:17:55.680+0000,2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/authors/OL10029831A,Bart At,null,1,2021-12-28T06:54:12.813+0000,2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/authors/OL10048132A,Keren Etkin,null,1,2021-12-28T22:25:37.697+0000,2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz
/authors/OL1006225A,Stephen McCafferty,Stephen McCafferty,2,2008-08-20T18:38:53.217+0000,2023-06-16 18:36:45.959,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_authors_2023-01-10.txt.gz


In [0]:
%sql
SELECT COUNT(*) FROM bronze_openlibrary.ol_authors --11316199

count(1)
11316199


In [0]:
%sql
SELECT COUNT(*) FROM silver_openlibrary.ol_authors --11316199

count(1)
11316199


#### Create table bronze_openlibrary.ol_works

In [0]:
%sql
SELECT * FROM bronze_openlibrary.ol_works LIMIT 5;

type,reference,revision_number,last_modified,json_col,ctl_created_timestamp,ctl_source,ctl_file_name
/type/work,/works/OL10000077W,7,2021-08-13T09:02:17.282+0000,"{""title"": ""Classic garden style"", ""subjects"": [""Floral decorations"", ""Garden ornaments and furniture"", ""Decoration and ornament"", ""Garden structures"", ""Jardins"", ""Meubles et ornements"", ""Antiquit\u00e9s (Objets anciens)""], ""key"": ""/works/OL10000077W"", ""authors"": [{""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL3965042A""}}], ""type"": {""key"": ""/type/work""}, ""covers"": [11604950], ""latest_revision"": 7, ""revision"": 7, ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2021-08-13T09:02:17.282451""}}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000238W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""H\u00e9rault de S\u00e9chelles, ou les infortunes de la beaut\u00e9"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3140482], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000238W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965269A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000261W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""Envoy\u00e9s sp\u00e9cieux"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3142759], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000261W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965289A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000387W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""JC de Castelbajac"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3140687], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000387W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965419A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz
/type/work,/works/OL10000507W,3,2010-04-28T06:54:19.472+0000,"{""title"": ""L'Alchimie de la gu\u00e9rison, tome 2"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:57:19.964652""}, ""covers"": [3140956], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10000507W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3965546A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}",2023-06-16 18:23:42.501,historical_load,dbfs:/mnt/cidaen-open-library/ol_dump_works_2023-01-10.txt.gz


In [0]:
df = spark.sql("SELECT * FROM bronze_openlibrary.ol_works")
schema = StructType(
    [
        StructField('title', StringType(), True),
        StructField('subjects', ArrayType(StringType(), True), True),
        StructField('created', StringType(), True),
        StructField('authors', StringType(), True),
        StructField('type', StringType(), True),
        StructField('last_modified', StringType(), True),
        StructField('revision', IntegerType(), True),
        StructField('description', StringType(), True)   
    ]
)

df = df.withColumn("json_col", from_json("json_col", schema))\
    .select(col('type'),col('reference'), col('revision_number'), col('json_col.*'))
df.createOrReplaceTempView("vw_silver_works_first_step")

In [0]:
%sql
SELECT reference,
       revision_number,
       last_modified,
       title,
       subjects,
       created,
       description,
       authors
FROM vw_silver_works_first_step;

In [0]:
# import re
# json_string = [{"type":{"key":"/type/author_role"},"author":{"key":"/authors/OL5057558A"}},{"type":{"key":"/type/author_role"},"author":{"key":"/authors/OL11645129A"}}]
# df_test = re.findall('/authors/OL\d+A',str(json_string))
# df_test

In [0]:
df = spark.sql("""SELECT reference,
                         revision_number,
                         last_modified,
                         title,
                         subjects,
                         created,
                         description,
                         authors
                    FROM vw_silver_works_first_step;""")

#schemas for unpacking some of the columns
created_schema = StructType(
    [
        StructField('type', StringType(), True),
        StructField('value', TimestampType(), True)
    ]
)

description_schema = StructType(
    [
        StructField('type', StringType(), True),
        StructField('value', StringType(), True)
    ]
)

last_modified_schema = StructType(
    [
        StructField('type', StringType(), True),
        StructField('value', TimestampType(), True)
    ]
)

description_str_schema = StructType([
    StructField("a_renamed", StringType()),
    StructField("b", StringType())])



df2 = df.withColumn("created", from_json("created", created_schema)) \
        .withColumn("description", from_json("description",description_schema)) \
        .withColumn("last_modified", from_json("last_modified",last_modified_schema)) \
        .withColumn('main_author', regexp_extract(col("authors"), r'/authors/OL\d+A', 0)) \
        .select(col('reference'), \
                col('title'), \
                col('main_author'), \
                col('revision_number'), \
                col('subjects'), \
                col('description.value').alias('description_text'), \
                col('created.value').alias('created_timestamp'), \
                col('last_modified.value').alias('last_modified_timestamp')
               )
df2.createOrReplaceTempView("vw_silver_works_second_step")

In [0]:
%sql
CREATE OR REPLACE TABLE silver_openlibrary.ol_works AS
SELECT DISTINCT * FROM vw_silver_works_second_step

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM silver_openlibrary.ol_works;

count(1)
30852119


In [0]:
%sql
SELECT * FROM silver_openlibrary.ol_works LIMIT 5

reference,title,main_author,revision_number,subjects,description_text,created_timestamp,last_modified_timestamp
/works/OL18042392W,"An duanaire: co-thional ùr de dh-órain, de dhuanagan, etc: Nach robh riabh ...",/authors/OL1239811A,1,null,null,2018-12-17T07:18:18.519+0000,2018-12-17T07:18:18.519+0000
/works/OL18065638W,"O descobrimento do Brazil, romance original",/authors/OL928893A,1,null,null,2018-12-18T08:52:38.563+0000,2018-12-18T08:52:38.563+0000
/works/OL18138077W,Lehrbuch der hebräischen Archäologie,/authors/OL1935382A,2,"List(Jews, Antiquities, Rites and ceremonies in the Bible, Bible)",null,2018-12-20T16:54:04.836+0000,2022-12-22T01:43:13.261+0000
/works/OL18160729W,"Proceedings of the FREENIX Track : 2000 USENIX Annual Technical Conference, June 18-23, 2000, San Diego, California, USA",/authors/OL7392206A,1,List(UNIX (Computer file)),null,2019-01-10T04:52:32.156+0000,2019-01-10T04:52:32.156+0000
/works/OL18234023W,The kids' catalog of Jewish holidays,/authors/OL19214A,5,"List(Juvenile fiction, Judaism, Handicraft, Fasts and feasts, Juvenile literature, Jewish Cookery, Jewish crafts, Jewish cookery, Jewish cooking)","Presents stories, poems, songs, recipes, crafts, and other activities for special days that are significant to Jews.",2019-02-04T13:42:31.516+0000,2022-02-26T17:06:12.311+0000


#### Create table silver_openlibrary.ol_readings

In [0]:
%sql
CREATE OR REPLACE TABLE silver_openlibrary.ol_readings AS
SELECT reference AS works_reference,
       YEAR(last_updated) AS flag_status_year,
       MONTH(last_updated) AS flag_status_month,
       read_status,
       COUNT(1) AS read_status_count
FROM bronze_openlibrary.ol_readings
GROUP BY 1, 2, 3, 4

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM silver_openlibrary.ol_readings;

count(1)
3984328


In [0]:
%sql
SELECT * FROM silver_openlibrary.ol_readings LIMIT 5

works_reference,flag_status_year,flag_status_month,read_status,read_status_count
/works/OL675443W,2018,1,Want to Read,2
/works/OL15827513W,2018,2,Want to Read,1
/works/OL2690357W,2018,4,Want to Read,2
/works/OL930555W,2018,3,Want to Read,1
/works/OL4325175W,2018,6,Want to Read,1


#### Create table silver_openlibrary.ol_ratings

In [0]:
%sql
CREATE OR REPLACE TABLE silver_openlibrary.ol_ratings AS
SELECT reference AS works_reference,
       YEAR(review_date) AS review_year,
       MONTH(review_date) AS review_month,
       COUNT(rating) AS rating_count,
       MEAN(rating) AS rating_avg
FROM bronze_openlibrary.ol_ratings 
GROUP BY 1, 2, 3

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM silver_openlibrary.ol_ratings;

count(1)
279883


In [0]:
%sql
SELECT * FROM silver_openlibrary.ol_ratings LIMIT 5

works_reference,review_year,review_month,rating_count,rating_avg
/works/OL2355832W,2019,1,1,5.0
/works/OL449557W,2019,10,1,3.0
/works/OL18164795W,2020,6,1,5.0
/works/OL21941236W,2022,4,1,5.0
/works/OL72398W,2019,11,1,2.0


## Gold

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold_openlibrary

### Create tables in Gold - Aggregations

#### Create table gold_openlibrary.ol_books

In [0]:
%sql
CREATE OR REPLACE TABLE gold_openlibrary.ol_books AS
SELECT w.reference AS works_reference,
       w.title,
       w.main_author AS author_reference,
       a.author_name,
       a.author_personal_name,
       w.revision_number,
       w.subjects,
       w.description_text,
       w.created_timestamp,
       w.last_modified_timestamp,
       COALESCE(CONCAT(YEAR(w.created_timestamp),MONTH(w.created_timestamp)),"190001") AS ctl_year_month
FROM silver_openlibrary.ol_works w
LEFT JOIN silver_openlibrary.ol_authors a
ON w.main_author = a.author_ref;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM gold_openlibrary.ol_books;

count(1)
30852119


In [0]:
%sql
SELECT * FROM gold_openlibrary.ol_books LIMIT 5

works_reference,title,author_reference,author_name,author_personal_name,revision_number,subjects,description_text,created_timestamp,last_modified_timestamp,ctl_year_month
/works/OL26542484W,Arbeitslosengeld 2 Für Geringverdiener und Erwerbslose,/authors/OL10000012A,Wenzel Gerd,null,1,null,null,2021-12-26T21:20:45.943+0000,2021-12-26T21:20:45.943+0000,202112
/works/OL26542539W,Six Suites,/authors/OL10000017A,Bach Johann Sebastian,null,1,null,null,2021-12-26T21:22:25.076+0000,2021-12-26T21:22:25.076+0000,202112
/works/OL26542631W,¿Dolor de Espalda y Rodilla?,/authors/OL10000083A,Evaristo Rodríguez Valverde D.P.,null,1,null,null,2021-12-26T21:23:32.493+0000,2021-12-26T21:23:32.493+0000,202112
/works/OL26543288W,Innovation Im Verwaltungsrecht Durch Internationalisierung,/authors/OL10000208A,Nicola Wiesinger,null,2,"List(Administrative law, Environmental law, Convention on Access to Information, Public Participation in Decision-making, and Access to Justice in Environmental Matters, Convention on Access to Information, Public Participation in Decision-making, and Access to Justice in Environmental Matters (1998))",null,2021-12-26T21:36:25.004+0000,2022-12-07T08:12:04.051+0000,202112
/works/OL26543683W,RECHTLICHEN GRENZEN DER ZULÄSSIGKEIT DES CASH POOLING,/authors/OL10000248A,Matthias Gärtner,null,1,null,null,2021-12-26T21:44:54.776+0000,2021-12-26T21:44:54.776+0000,202112


#### Create table gold_openlibrary.ol_reviews

In [0]:
%sql
CREATE OR REPLACE TABLE gold_openlibrary.ol_reviews AS
SELECT rd.works_reference, 
       rd.flag_status_year AS review_year, 
       rd.flag_status_month AS review_month,
       rt.rating_count,
       rt.rating_avg,
       rd.read_status,
       rd.read_status_count,
       COALESCE(CONCAT(rd.flag_status_year,rd.flag_status_month),"190001") AS ctl_year_month
FROM silver_openlibrary.ol_readings rd
LEFT JOIN silver_openlibrary.ol_ratings rt
ON rd.works_reference = rt.works_reference
AND rd.flag_status_year = rt.review_year
AND rd.flag_status_month = rt.review_month

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM gold_openlibrary.ol_reviews

count(1)
3984328


In [0]:
%sql
SELECT * FROM gold_openlibrary.ol_reviews LIMIT 5

works_reference,review_year,review_month,rating_count,rating_avg,read_status,read_status_count,ctl_year_month
/works/OL675443W,2018,1,null,null,Want to Read,2,20181
/works/OL15827513W,2018,2,null,null,Want to Read,1,20182
/works/OL2690357W,2018,4,null,null,Want to Read,2,20184
/works/OL930555W,2018,3,null,null,Want to Read,1,20183
/works/OL4325175W,2018,6,null,null,Want to Read,1,20186


#### Create table gold_openlibrary.works_with_reviews_summary

In [0]:
%sql
CREATE OR REPLACE TABLE gold_openlibrary.works_with_reviews_summary AS
SELECT r.works_reference,
       r.review_year,
       r.review_month,
       r.rating_count,
       r.rating_avg,
       b.title,
       b.author_name,
       b.author_personal_name,
       b.subjects,
       b.description_text,
       b.ctl_year_month
FROM silver_openlibrary.ol_ratings r 
INNER JOIN gold_openlibrary.ol_books b 
ON r.works_reference = b.works_reference

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT COUNT(*) FROM gold_openlibrary.works_with_reviews_summary;

count(1)
268995


In [0]:
%sql
SELECT * FROM gold_openlibrary.works_with_reviews_summary LIMIT 5;

works_reference,review_year,review_month,rating_count,rating_avg,title,author_name,author_personal_name,subjects,description_text,ctl_year_month
/works/OL4772459W,2022,3,1,5.0,Lulu in Hollywood,Louise Brooks,Louise Brooks,"List(Biography, Motion picture actors and actresses, Moving-picture actors and actresses, Films, Actresses)","Eight autobiographical essays by Brooks, on topics ranging from her childhood in Kansas and her early days as a Denishawn and Ziegfeld Follies dancer to her friendships with Martha Graham, Charles Chaplin, W. C. Fields, Humphrey Bogart, and others are collected here.",200912
/works/OL26617158W,2022,6,1,5.0,Extragalaktisch Band 2 - Weltenwandler,Yasmin Kurul Mendoza,null,"List(Alien, Außerirdische, Merlin, Kinderbuch, vegane Literatur, psy-fiction, sci-fi, galaktische Föderation)","Die Reise geht weiter! Lubi Golder Morgen hat es als stotternder Junge nicht leicht. Zu allem Überfluss wird er neuerdings auch noch von merkwürdigen Träumen heimgesucht, die sich bald nicht mehr von der Realität unterscheiden lassen. Als er eines Morgens wie durch ein Wunder von seinem Stottern geheilt ist, beginnt er zu verstehen, dass die Wach-Welt und die Traum-Welt eng miteinander verknüpft sind. Er beginnt seine Fähigkeit des Traumreisens zu trainieren und kurze Zeit später findet er sich wieder in der Bibliothek der Marmaroth-Akademie, mit einem der größten Meister aller Zeiten… Inhalt: •„Extragalaktisch Band 2 – Weltenwandler“, •inklusive Rückblick aus Band 1, sodass Band 2 direkt gelesen werden kann, ohne vorab den vorigen Band gelesen haben zu müssen; •2 köstliche vegane Weihnachts-Backrezepte!",202112
/works/OL4780288W,2020,3,1,5.0,Richard Morris Hunt,Paul R. Baker,Paul R. Baker,"List(Biography, Architects, Hunt, richard morris, 1828-1895, Architectes, Biographies)",null,200912
/works/OL4793911W,2021,2,1,1.0,Passover,Fumiko Kometani,Fumiko Kometani,"List(Fiction, general)",null,200912
/works/OL4801345W,2022,11,1,4.0,Pronouns and people,Peter Mühlhäusler,Peter Mühlhäusler,"List(Comparative and general Grammar, Pronoun, Sapir-Whorf hypothesis, Sociolinguistics, Grammar, comparative and general)",null,200912


#### Create table gold_openlibrary.ol_works_summary

In [0]:
%sql
CREATE OR REPLACE TABLE gold_openlibrary.ol_works_summary AS
SELECT b.works_reference,
       b.title,
       b.author_name,
       b.author_personal_name,
       b.revision_number,
       b.subjects,
       b.description_text,
       COUNT(rating) AS rating_count,
       MEAN(rating) AS rating_avg,
       FLOOR(20100 + rand()*100) AS ctl_year_month
FROM gold_openlibrary.ol_books b
LEFT JOIN bronze_openlibrary.ol_ratings rt 
ON rt.reference = b.works_reference
LEFT JOIN bronze_openlibrary.ol_readings rd
ON rd.reference = b.works_reference
GROUP BY 1, 2, 3, 4, 5, 6, 7;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold_openlibrary.ol_works_summary LIMIT 5

works_reference,title,author_name,author_personal_name,revision_number,subjects,description_text,rating_count,rating_avg,ctl_year_month
/works/OL1000013W,The prairie chicken kill,Bill Crider,Bill Crider,9,"List(Fiction, Private investigators, Truman Smith (Fictitious character))",null,0,null,20110
/works/OL10000178W,"Le Mariage de Figaro, de Beaumarchais",S. Kefallonitis,null,3,null,null,0,null,20184
/works/OL10000575W,Urgences vitales prises en charge et diagnostics,Plaisance,null,4,"List(Médecine d'urgence, Urgences médicales)",null,0,null,20105
/works/OL10000823W,"La Grande Galerie Des Peintures Musee Du Louvre, D'Orsay, Centre Pompidou/Musee National D'Art Moderne",Francoise Bertaux,null,2,"List(Painting, Catalogs, Peinture, Thèmes, motifs, Catalogues, Musée du Louvre, Musée d'Orsay, Centre Georges Pompidou)",null,0,null,20107
/works/OL10000827W,Access 2000,Jean Nashe,null,3,null,null,0,null,20118


In [0]:
%sql
--checking if the distribution is good enough for partitioning the table export
SELECT ctl_year_month, COUNT(1) 
FROM gold_openlibrary.ol_works_summary
GROUP BY 1
--approximately 300K records per bucket. Looks good.

ctl_year_month,count(1)
20166,309009
20144,308752
20195,308578
20130,308367
20123,307925
20103,309064
20116,307551
20191,307405
20199,308800
20193,308650


## Persisting data. Uploading parquet files to S3

### widgets configuration

In [0]:
# ['bronze_openlibrary.ol_ratings',
# 'bronze_openlibrary.ol_readings',
# 'bronze_openlibrary.ol_works',
# 'bronze_openlibrary.ol_authors',
# 'silver_openlibrary.ol_ratings',
# 'silver_openlibrary.ol_readings',
# 'silver_openlibrary.ol_works',
# 'silver_openlibrary.ol_authors',
# 'gold_openlibrary.ol_books',
# 'gold_openlibrary.ol_reviews',
# 'gold_openlibrary.books_summary']

In [0]:
dbutils.widgets.text("access_key", "")
dbutils.widgets.text("secret_access_key", "")
dbutils.widgets.text("account_id", "")
dbutils.widgets.text("s3_bucket", "")
dbutils.widgets.dropdown("table", "gold_openlibrary.ol_books", 
             ['gold_openlibrary.ol_books',
              'gold_openlibrary.ol_reviews',
              'gold_openlibrary.works_with_reviews_summary',
              'gold_openlibrary.ol_works_summary'])

### Imports

In [0]:
# this can be installed in cluster

In [0]:
pip install s3fs

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import os
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import Table

### Function - Loading files to S3

In [0]:
bucket_name = dbutils.widgets.get("s3_bucket")
aws_access_key_id= dbutils.widgets.get("access_key")
aws_secret_access_key= dbutils.widgets.get("secret_access_key")
region_name='eu-central-1'
endpoint_url= f"https://s3.eu-central-1.amazonaws.com/{bucket_name}"

tbl = dbutils.widgets.get("table")

fs = s3fs.S3FileSystem(
anon=False,
use_ssl=True,
    client_kwargs={
        "region_name": region_name,
        "endpoint_url": endpoint_url,
        "aws_access_key_id": aws_access_key_id,
        "aws_secret_access_key": aws_secret_access_key,
        "verify": True,
    }
)

def load_files_to_s3(tbl: str, where_clause: str):
    #
    # WHERE clause for loading in batches and expects the following format: YYYYMM
    #
    try:
        
        file_name = tbl.replace(".","_")+".csv"
        s3_filepath = file_name.split(".")[0]
        
        query = f"SELECT * FROM {tbl} WHERE ctl_year_month = {where_clause}"
        df = spark.sql(query)
        pd_df = df.toPandas()

        print("LOADING FILES TO S3...")

        pq.write_to_dataset(
            Table.from_pandas(pd_df),
            s3_filepath,
            filesystem=fs,
            use_dictionary=True,
            partition_cols=['ctl_year_month'],
            compression="snappy",
            version="2.4",
)
    except ClientError as e:
        print(e)
    finally:
        print("ALL FILES UPLOADED TO S3")

#load selected table to S3
query_part = f"SELECT DISTINCT ctl_year_month FROM {tbl}"
partitions = list(spark.sql(query_part).toPandas())

for where_clause in partitions:
    load_files_to_s3(tbl,where_clause)

LOADING FILES TO S3...
ALL FILES UPLOADED TO S3


## Analysis of the dataset

In [0]:
%sql
-- 20 authors with more published works
SELECT author_name,       
       COUNT(works_reference)
FROM gold_openlibrary.ol_books
WHERE author_reference IS NOT NULL
AND (author_name IS NOT NULL OR author_personal_name IS NOT NULL)
GROUP BY 1
ORDER BY COUNT(works_reference) DESC 
LIMIT 20

author_name,count(works_reference)
Great Britain,61094
United States. Congress. House,36308
Philip M. Parker,33343
Rand McNally,29519
ICON Group Ltd.,27877
n/a,25418
United States,25066
Jeryx Publishing,23016
NA,21859
No name,16722


Philip M. Parker is an American economist and academic, currently the INSEAD Chaired Professor of Management Science at INSEAD in Fontainebleau, France. He has patented a method to automatically produce a set of similar books from a template which is filled with data from database and Internet searches.

Rand McNally is an American technology and publishing company that provides mapping, software and hardware for consumer electronics, commercial transportation and education markets. The company is headquartered in Chicago, with a distribution center in Richmond, Kentucky.

In [0]:
%sql
SELECT * FROM gold_openlibrary.works_with_reviews_summary LIMIT 10

works_reference,review_year,review_month,rating_count,rating_avg,title,author_name,author_personal_name,subjects,description_text,ctl_year_month
/works/OL4772459W,2022,3,1,5.0,Lulu in Hollywood,Louise Brooks,Louise Brooks,"List(Biography, Motion picture actors and actresses, Moving-picture actors and actresses, Films, Actresses)","Eight autobiographical essays by Brooks, on topics ranging from her childhood in Kansas and her early days as a Denishawn and Ziegfeld Follies dancer to her friendships with Martha Graham, Charles Chaplin, W. C. Fields, Humphrey Bogart, and others are collected here.",200912
/works/OL26617158W,2022,6,1,5.0,Extragalaktisch Band 2 - Weltenwandler,Yasmin Kurul Mendoza,null,"List(Alien, Außerirdische, Merlin, Kinderbuch, vegane Literatur, psy-fiction, sci-fi, galaktische Föderation)","Die Reise geht weiter! Lubi Golder Morgen hat es als stotternder Junge nicht leicht. Zu allem Überfluss wird er neuerdings auch noch von merkwürdigen Träumen heimgesucht, die sich bald nicht mehr von der Realität unterscheiden lassen. Als er eines Morgens wie durch ein Wunder von seinem Stottern geheilt ist, beginnt er zu verstehen, dass die Wach-Welt und die Traum-Welt eng miteinander verknüpft sind. Er beginnt seine Fähigkeit des Traumreisens zu trainieren und kurze Zeit später findet er sich wieder in der Bibliothek der Marmaroth-Akademie, mit einem der größten Meister aller Zeiten… Inhalt: •„Extragalaktisch Band 2 – Weltenwandler“, •inklusive Rückblick aus Band 1, sodass Band 2 direkt gelesen werden kann, ohne vorab den vorigen Band gelesen haben zu müssen; •2 köstliche vegane Weihnachts-Backrezepte!",202112
/works/OL4780288W,2020,3,1,5.0,Richard Morris Hunt,Paul R. Baker,Paul R. Baker,"List(Biography, Architects, Hunt, richard morris, 1828-1895, Architectes, Biographies)",null,200912
/works/OL4793911W,2021,2,1,1.0,Passover,Fumiko Kometani,Fumiko Kometani,"List(Fiction, general)",null,200912
/works/OL4801345W,2022,11,1,4.0,Pronouns and people,Peter Mühlhäusler,Peter Mühlhäusler,"List(Comparative and general Grammar, Pronoun, Sapir-Whorf hypothesis, Sociolinguistics, Grammar, comparative and general)",null,200912
/works/OL1036525W,2021,7,1,5.0,Sindh,"Khan, Mubarak Ali.","Khan, Mubarak Ali.",List(History),"Historical study of Sindh, Pakistan, from the early Islamic period to date.",200912
/works/OL27036094W,2022,10,1,5.0,Srimad Bhagwat Geeta in Hindi,Baal Gangadhar Tilak,null,null,null,20221
/works/OL4815643W,2021,1,1,5.0,Knitting Without Tears,Elizabeth Zimmermann,Elizabeth Zimmermann,"List(Knitting, Stickning, Praktiska handledningar, Knit goods, Handbooks, manuals, Care, Sweaters)",null,200912
/works/OL4828337W,2021,9,1,3.0,Synge and Anglo-Irish literature,Daniel Corkery,Daniel Corkery,null,null,200912
/works/OL27295611W,2022,10,1,5.0,The Case of the Missing Bicycles,Milo Stone,null,List(Children's fiction),null,20222


In [0]:
%sql
-- Top rated books with more than 15 reviews
SELECT works_reference, 
       title, 
       author_name, 
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
GROUP BY 1,2,3
HAVING rating_count > 15
ORDER BY 4 DESC, 5 DESC 
LIMIT 50

works_reference,title,author_name,rating_average,rating_count
/works/OL17427523W,The Brilliant World Of Tom Gates,Liz Pichon,5.0,16
/works/OL17610987W,The Baby-Sitter's Club,Raina Telgemeier,4.94,17
/works/OL18289083W,"Faith, Keyes, and Clark's Industrial chemicals",William Lawrence Faith,4.88,16
/works/OL21640039W,The Psychology of Money,Morgan Housel,4.88,16
/works/OL262554W,Sherlock Holmes (Adventures of Sherlock Holmes / Case-Book of Sherlock Holmes / His Last Bow / Hound of the Baskervilles / Memoirs of Sherlock Holmes / Return of Sherlock Holmes / Sign of Four / Study in Scarlet / Valley of Fear),Arthur Conan Doyle,4.85,26
/works/OL3004224W,The essential Calvin and Hobbes,Bill Watterson,4.82,17
/works/OL16361049W,Keeper of the Lost Cities,Shannon Messenger,4.79,29
/works/OL15946464W,The Throne of Fire,Rick Riordan,4.77,22
/works/OL547371W,A bad case of stripes,David Shannon,4.76,17
/works/OL20893680W,Piranesi,Susanna Clarke,4.75,20


In [0]:
%sql
-- Bottom rated books with more than 15 reviews
SELECT works_reference, 
       title, 
       author_name, 
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
GROUP BY 1,2,3
HAVING rating_count > 15
ORDER BY 4 ASC, 5 DESC 
LIMIT 50

works_reference,title,author_name,rating_average,rating_count
/works/OL9265172W,Sleeping with a Stranger,Anne Mather,2.69,16
/works/OL3845170W,Farewell to Love,Jessica Steele,2.88,16
/works/OL20907281W,Ready Player Two,Ernest Cline,3.0,17
/works/OL45793W,Charlie and the Great Glass Elevator,Roald Dahl,3.1,29
/works/OL4342659W,In Name Only,Roberta Leigh,3.12,17
/works/OL17358374W,Yes Please,Amy Poehler,3.13,16
/works/OL11327982W,Blackmail,Penny Jordan,3.13,16
/works/OL2001062W,The Namesake,Jhumpa Lahiri,3.13,16
/works/OL46913W,Congo,Michael Crichton,3.14,22
/works/OL547172W,Uglies (Uglies #1),Scott Westerfeld,3.14,22


In [0]:
%sql
-- Most rated books (review counts)
SELECT works_reference, 
       title, 
       author_name, 
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
GROUP BY 1,2,3
ORDER BY 5 DESC 
LIMIT 50

works_reference,title,author_name,rating_average,rating_count
/works/OL82563W,Harry Potter and the Philosopher's Stone,J. K. Rowling,4.27,419
/works/OL17590212W,The Subtle Art of Not Giving a F*ck,Mark Manson,3.9,344
/works/OL18020194W,It Ends With Us,Colleen Hoover,4.11,318
/works/OL81613W,It,Stephen King,4.14,292
/works/OL1168007W,Animal Farm,George Orwell,4.07,288
/works/OL262758W,The Hobbit,J.R.R. Tolkien,4.25,276
/works/OL82536W,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,4.3,272
/works/OL17930368W,Atomic Habits,James Clear,4.05,256
/works/OL5735363W,The Hunger Games,Suzanne Collins,4.04,246
/works/OL82537W,Harry Potter and the Chamber of Secrets,J. K. Rowling,4.2,229


In [0]:
%sql
-- Top 100 rated authors with over 10 votes
SELECT author_name, 
       COUNT(title) AS number_of_publications,
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
GROUP BY 1
HAVING rating_count > 10
ORDER BY 3 DESC 
LIMIT 100

-- we can observe that most of these authors just have a few ratings, so we can increase the number of votes to be considered

author_name,number_of_publications,rating_average,rating_count
Ranjot Singh Chahal,16,5.0,16
Paul Procter,10,5.0,12
alexis karpouzos,13,5.0,13
Jane Roberts,16,5.0,16
Julie Ann Walker,13,5.0,13
Aleksandra Ziolkowska-Boehm,13,5.0,13
Milena Tsvetkova,11,5.0,11
Thời tiết 24h,34,5.0,34
Valdivânia Albuquerque do Nascimento,13,5.0,13
John Eastwood,10,5.0,12


In [0]:
%sql
-- Top 100 authors with more than 25 votes
SELECT author_name, 
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
GROUP BY 1
HAVING rating_count > 25
ORDER BY 2 DESC 
LIMIT 100

author_name,rating_average,rating_count
Jean-Pierre van Rossem,5.0,27
Dr. Andrej Poleev,5.0,31
Hashem Ibraheem Felaly,5.0,40
Elspeth Campbell Murphy,5.0,30
Thời tiết 24h,5.0,34
Cat Ellington,5.0,31
Paul Galdone,4.99,77
Mallory Kane,4.93,41
Robert D. San Souci,4.9,49
Barbara Pym,4.9,30


In [0]:
%sql
-- What are the top rated books from the top 100 rated authors
WITH top_authors AS (
SELECT author_name, 
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
GROUP BY 1
HAVING rating_count > 25
ORDER BY 2 DESC 
LIMIT 100)

SELECT author_name, 
       title,
       subjects,
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE author_name IN (SELECT author_name FROM top_authors)
AND rating_count > 5
GROUP BY 1, 2, 3
ORDER BY 4 DESC;

author_name,title,subjects,rating_average,rating_count
Shannon Messenger,Keeper of the Lost Cities,"List(Ability, Fiction, Fantasy, Psychic ability, Identity, Memory, Foster parents, Elves, Children's fiction, Fairies, fiction, Fantasy fiction, Parapsychology, fiction, Identity, fiction, Juvenile fiction, nyt:series-books=2016-11-20, New York Times bestseller, Identity (Philosophical concept), Action & Adventure, Fantasy & Magic, Social Themes, Friendship, Identity (Psychology), Foster home care, Adventure and adventurers, fiction, Friendship, fiction)",4.86,7
Soman Chainani,The School for Good and Evil,"List(Magic, Juvenile fiction, Fiction, Fairy Schools, Schools, Identity (Psychology) in children, Friendship, Good and evil, Fantasy, Children's fiction, Friendship, fiction, Good and evil, fiction, Fantasy fiction, Schools, fiction, School stories, Handbooks, manuals, Juvenile literature, Characters, American Fantasy fiction, Characters and characteristics, Children's stories, nyt:childrens-middle-grade=2013-06-02, New York Times bestseller)",4.83,6
Elisabetta Dami,The Kingdom of Fantasy,"List(Stilton, Geronimo, Fantasy, Juvenile fiction, Geronimo Stilton (Fictitious character), Fairies, Fiction, Children's fiction, Fantasy fiction, Stilton, geronimo (fictitious character), fiction)",4.75,12
Alice Oseman,"Heartstopper, Volume 1","List(Young adult fiction, comics & graphic novels, lgbt, Young adult fiction, comics & graphic novels, lgbtq+, High schools, Comic books, strips, Boys, Gays, Love in adolescence, Friendship, Mental illness, nyt:graphic-books-and-manga=2022-05-08, New York Times bestseller)",4.5,30
Victoria Jamieson,Roller Girl,"List(Roller skating, Juvenile fiction, Graphic novels, Fiction, Roller derby, Best friends, Comic books, strips, Friendship, Newbery Honor, Comic books, strips, etc., Friendship, fiction, Roller skating, fiction, Children's fiction, Patinaje sobre ruedas, Libros de caricaturas, tiras comicas, Amistad, Ficcion juvenil, Spanish language materials, Cartoons and comics, Mejores amigos, nyt:paperback-graphic-books=2015-04-12, New York Times bestseller)",4.5,6
Art Spiegelman,The Complete Maus,"List(Comics & graphic novels, general, Holocaust survivors, Children of holocaust survivors, Holocaust, jewish (1939-1945), Long Now Manual for Civilization, Graphic novels, Fathers and sons, Comic books, strips, Biography, Nazis, Jewish Personal narratives, Holocaust, Cartoons and comics, Survivors, Judenvernichtung, World War, 1939-1945, History, 20th Century, Parent-Child Relations, Jews, Überlebender, Pariser Friedenskonferenz 1919-1920 Paris Polish Delegation gnd, Holocaust, jewish (1939-1945), personal narratives, Comics & graphic novels, nonfiction, general, World war, 1939-1945, personal narratives, jewish, nyt:hardcover-graphic-books=2012-08-19, New York Times bestseller, Graphic novel, Comicbooks, strips, Collections from individual artists, American strip cartoons, nyt:paperback_graphic_books=2011-09-17, open_syllabus_project, Bandes dessinées, Biographies, Holocauste, 1939-1945, Juifs, Poland, Survivants de l'Holocauste, United States, Comics & graphic novels, horror, Comics & graphic novels, literary, Comics & graphic novels, religious, Comics & graphic novels, nonfiction, biography & memoir, collectionID:MausBan, Banned books, Holocaust, Jewish (1939-1945), Comic)",4.45,11
Phil Knight,Shoe Dog,"List(Businesspeople, juvenile literature, Business, juvenile literature, Business, biography, nyt:sports=2017-11-12, New York Times bestseller, nyt:paperback-nonfiction=2018-05-20, Sporting goods industry, Nike (Firm), Businesspeople, Biography, History, Businesspeople, biography, Business enterprises, history, nyt:business-books=2016-05-08, memoir, nike, Business, Sporting goods, BIOGRAPHY & AUTOBIOGRAPHY, Personal Memoirs, SPORTS & RECREATION, Running & Jogging, Unternehmer, Sportartikelindustrie, Nike Inc, Umschulungswerkstätten für Siedler und Auswanderer)",4.44,9
Trevor Noah,Born a Crime,"List(aparthe

In [0]:
%sql
-- Top rated books with a description available
SELECT title, 
       description_text,
       round(try_divide(SUM(rating_avg * rating_count), SUM(rating_count)),2) AS rating_average,
       SUM(rating_count) AS rating_count
FROM gold_openlibrary.works_with_reviews_summary
WHERE title IS NOT NULL
AND description_text IS NOT NULL
GROUP BY 1, 2
HAVING rating_count > 15
ORDER BY 3 DESC 
LIMIT 100

title,description_text,rating_average,rating_count
The Brilliant World Of Tom Gates,a comical novel,5.0,16
The essential Calvin and Hobbes,Includes cartoons from *Calvin and Hobbes* and *Something Under the Bed Is Drooling*,4.82,17
Wings of Fire,"Deep in the rain forest, danger awaits... Glory knows the dragon world is wrong about her tribe. After all, she isn't ""as lazy as a RainWing"" -- she isn't lazy at all! Maybe she wasn't meant to be one of the dragonets of destiny, as the older dragons constantly remind her, but Glory is sharp and her venom is deadly... except, of course, no one knows it. When the dragonets seek shelter in the rain forest, Glory is devastated to find that the treetops are full of RainWings that no dragon could ever call dangerous. They nap all day and know nothing of the rest of Pyrrhia. Worst of all, they don't realize -- or care -- that RainWings are going missing from their beautiful forest. But Glory and the dragonets are determined to find the missing dragons, even if it drags the peaceful RainWing kingdom where they never wanted to be -- in the middle of the war.",4.75,16
Piranesi,"**From the *New York Times* bestselling author of *Jonathan Strange & Mr. Norrell*, an intoxicating, hypnotic new novel set in a dreamlike alternative reality.** Piranesi's house is no ordinary building; its rooms are infinite, its corridors endless, its walls are lined with thousands upon thousands of statues, each one different from all the others. Within the labyrinth of halls an ocean is imprisoned; waves thunder up staircases, rooms are flooded in an instant. But Piranesi is not afraid; he understands the tides as he understands the pattern of the labyrinth itself. He lives to explore the house. There is one other person in the house--a man called The Other, who visits Piranesi twice a week and asks for help with research into A Great and Secret Knowledge. But as Piranesi explores, evidence emerges of another person, and a terrible truth begins to unravel, revealing a world beyond the one Piranesi has always known. For readers of Neil Gaiman's *The Ocean at the End of the Lane* and fans of Madeline Miller's *Circe*, *Piranesi* introduces an astonishing new world, an infinite labyrinth full of startling images of surreal beauty, haunted by the tides and the clouds. This description comes from the publisher.",4.75,20
Thea Stilton and the Lost Letters,The Thea Sisters are in Russia and they find an unexpected mystery when a robbery happens. The Thea Sisters investigate until they almost give up hope when they find something that gives them a breakthrough.,4.75,20
Harry Potter (series) 1-7,"The Harry Potter books throw you into an amazing fantasy world of witches and wizards, spells, magical creatures, He-Who-Must-Not-Be-Named, and a school of witchcraft and wizardry called Hogwarts. In this school Harry Potter takes many magical classes, plays Quidditch (the sport where you are on a broomstick scoring goals through hoops, and trying to find the magical snitch.), and defeats Voldemort. When Voldemort comes back at him again and again, Harry realizes that Voldemort is invincible, and there is only one way to kill him. With his lessons with Professor Dumbledore, he discovers Voldemort's past, who he was before he killed all those people, why he killed all those people (including Harry Potter's parents), and how he is still alive, but is almost a creature, with snake eyes, and slits for nostrils. This best-selling series is astonishing and breathtaking and will change your life forever, just by reading it.",4.74,27
"A child called ""it""","This book chronicles the unforgettable account of one of the most severe child abuse cases in California history. It is the story of Dave Pelzer, who was brutally beaten and starved by his emotionally unstable, alcoholic mother: a mother who played tortuous, unpredictable games--games that left him nearly dead. He had to learn how to play his mother's games in order to survive because